# Download Video

In [ ]:
# @title 🍪 [Manual] Upload Cookies
# @markdown - Upload file cookies (Untuk download twitter yang membutuhkan auth)
from google.colab import files
import os

print("📤 Silakan upload file cookies.txt (hasil dari Kiwi Browser)...")
uploaded = files.upload()

# Temukan file .txt yang diupload
cookies_file = None
for name in uploaded.keys():
    if name.endswith(".txt"):
        cookies_file = name
        break

# Rename ke cookies.txt
if cookies_file and os.path.exists(cookies_file):
    if cookies_file != "cookies.txt":
        os.rename(cookies_file, "cookies.txt")
        print(f"📁 File '{cookies_file}' di-rename menjadi 'cookies.txt'")
    else:
        print("✅ File sudah bernama cookies.txt")

    # Tampilkan contoh isi file
    print("📄 Contoh isi cookies.txt:")
    with open("cookies.txt", "r", encoding="utf-8") as f:
        for i in range(3):
            line = f.readline()
            if line:
                print("  ", line.strip())
else:
    print("❌ File .txt tidak valid ditemukan atau gagal upload.")

In [ ]:
# @title 🔗 [A1] Video Downloader (non-twitter)
# @markdown - Nama file boleh kosong

download_type = "auto"  # @param ["auto", "google_drive", "direct", "m3u8"]
video_url = ""  # @param {type:"string"}
file_name = ""  # @param {type:"string"}

from datetime import datetime
import os

# 🔧 Lokasi penyimpanan video
video_dir = "/content/video_downloader/video"
os.makedirs(video_dir, exist_ok=True)

# 🕒 Nama file default jika kosong
if not file_name.strip():
    timestamp = datetime.now().strftime("video_%Y%m%d_%H%M%S")
    file_name = f"{timestamp}.mp4"
elif not file_name.endswith(".mp4"):
    file_name += ".mp4"

# 📍 Path output final
output_path = os.path.join(video_dir, file_name)

# 🧾 Info
print(f"🎯 Link: {video_url}")
print(f"🧩 Jenis Unduhan: {download_type}")
print(f"📁 File akan disimpan di: {output_path}")

# 📌 Lanjutkan di Cell berikutnya berdasarkan tipe: gdown / aria2 / yt-dlp / m3u8

In [ ]:
# @title 🚀 [A2] Proses Download Berdasarkan Jenis yang Dipilih
# @markdown 👈 klik untuk mengunduh

import subprocess, re, os, time

print("🎯 Link:", video_url)
print("🧩 Jenis Unduhan:", download_type)
print("📁 Output:", output_path)
print("────────────────────────────")

def extract_drive_id(url):
    patterns = [
        r"drive\.google\.com\/file\/d\/([^\/]+)",
        r"drive\.google\.com\/open\?id=([^&]+)",
        r"drive\.google\.com\/uc\?id=([^&]+)",
    ]
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

def is_m3u8(url):
    return url.endswith(".m3u8") or ".m3u8?" in url

def is_drive(url):
    return "drive.google.com" in url

# 🔧 Pastikan dependensi
print("🔧 Memastikan dependensi...")
!apt -qq install -y ffmpeg
!pip install -q --upgrade yt-dlp gdown
!pip install -q --user m3u8downloader

# Tambahkan ~/.local/bin ke PATH (jika belum ada)
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.local/bin")

# 🌐 Tentukan alat berdasarkan input user atau auto-detect
tool = download_type

if tool == "auto":
    if is_drive(video_url):
        tool = "google_drive"
    elif is_m3u8(video_url):
        tool = "m3u8"
    else:
        tool = "direct"

print(f"🚀 Menggunakan alat: {tool}")

start_time = time.time()

try:
    if tool == "google_drive":
        print("📁 Mendownload dari Google Drive...")
        import gdown
        drive_id = extract_drive_id(video_url)
        if not drive_id:
            raise ValueError("❌ Tidak dapat mendeteksi ID file dari URL Google Drive.")
        gdown.download(f"https://drive.google.com/uc?id={drive_id}", output_path, quiet=False)

    elif tool == "m3u8":
        print("📺 Mendownload dari M3U8 menggunakan m3u8downloader...")
        downloader_path = os.path.expanduser("~/.local/bin/downloadm3u8")
        if not os.path.exists(downloader_path):
            raise FileNotFoundError("❌ 'downloadm3u8' tidak ditemukan di ~/.local/bin/. Coba restart runtime setelah install.")

        cmd = [
            downloader_path,
            "-o", output_path,
            video_url
        ]
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in process.stdout:
            print(f"\r{line.strip()[:150]}", end="", flush=True)
        process.wait()

    elif tool == "direct":
        print("📥 Mendownload dari Direct Link menggunakan yt-dlp...")
        cmd = [
            "yt-dlp",
            "-f", "best",
            "-o", output_path,
            "--no-warnings",
            "--retries", "3",
            video_url
        ]
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in process.stdout:
            print(f"\r{line.strip()[:150]}", end="", flush=True)
        process.wait()

    else:
        raise ValueError("❌ Jenis download tidak dikenali!")

    # ✅ Cek hasil
    if os.path.exists(output_path):
        size = os.path.getsize(output_path) / (1024 * 1024)
        elapsed = time.time() - start_time
        print(f"\n\n✅ Selesai! File disimpan di: {output_path}")
        print(f"📦 Ukuran file: {size:.2f} MB")
        print(f"⏱️ Waktu download: {elapsed:.2f} detik")
    else:
        print("\n⚠️ Download selesai tapi file tidak ditemukan.")

except Exception as e:
    print(f"\n❌ Terjadi kesalahan saat mengunduh: {e}")

In [ ]:
# @title 🎞️ [B] Unduh Video Twitter
# @markdown - Masukkan URL tweet (single-video / multi-video)

# ✅ Install yt-dlp (jika belum)
!pip install -U yt-dlp > /dev/null

tweet_url = ""  # @param {type: "string"}

import os, re, json, glob, subprocess, datetime
from tqdm import tqdm

# 📁 Gunakan folder video_downloader
base_dir = "/content/video_downloader"
video_dir = os.path.join(base_dir, "video")
output_dir = os.path.join(base_dir, "output")
os.makedirs(video_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# 🆔 Ambil Tweet ID
match = re.search(r"/status/(\d+)", tweet_url)
if not match:
    raise ValueError("❌ URL tidak valid: Tidak ditemukan Tweet ID.")
tweet_id = match.group(1)

# 🔍 Simulasi metadata
info = None
ydl_opts = {
    "quiet": True,
    "simulate": True,
    "extract_flat": True,
    "dump_single_json": True,
}
print(f"🔍 Mendeteksi metadata tweet... ({tweet_url})")

try:
    import yt_dlp
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(tweet_url, download=False)
except Exception as e:
    print(f"❌ Gagal mendeteksi metadata: {e}")
    if "authentication" in str(e).lower():
        print("🔐 Tweet ini kemungkinan membutuhkan cookies.txt (login).")

# 💾 Simpan JSON jika tersedia
if info:
    json_path = os.path.join(output_dir, f"deteksi_tweet_{tweet_id}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(info, f, ensure_ascii=False, indent=2)
    print(f"📄 Metadata disimpan ke: {json_path}")
else:
    print("⚠️ Metadata tidak tersedia atau tidak disimpan.")

# 🍪 Cek cookies
use_cookies = os.path.exists("cookies.txt")
print("🔐 Menggunakan cookies.txt" if use_cookies else "🔓 Tidak menggunakan cookies")

# 📥 Proses download (progres dengan tqdm)
print("📥 Mulai mengunduh video...\n")
command = ["yt-dlp"]
if use_cookies:
    command += ["--cookies", "cookies.txt"]
command += [
    "-f", "best",
    "-o", f"{video_dir}/%(id)s_video.%(ext)s",
    tweet_url
]

progress_bar = tqdm(total=100, desc="📥 Download", unit="%")
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in process.stdout:
    line = line.strip()
    if "%" in line:
        match = re.search(r'(\d{1,3}\.\d)%', line)
        if match:
            percent = float(match.group(1))
            progress_bar.n = int(percent)
            progress_bar.refresh()
    elif "[download]" in line or "Destination" in line:
        print(line)
process.wait()
progress_bar.n = 100
progress_bar.refresh()
progress_bar.close()

print("✅ Download selesai.")

# 📦 Status akhir
downloaded = glob.glob(os.path.join(video_dir, "*_video.*"))
total_size_mb = sum(os.path.getsize(f) for f in downloaded) / (1024 * 1024)
file_names = [os.path.basename(f) for f in downloaded]

print("\n📊 Ringkasan Status:")
print(f"- 📌 URL Tweet       : {tweet_url}")
print(f"- 🆔 ID Tweet        : {tweet_id}")
print(f"- 🔐 Cookies         : {'✅ Digunakan' if use_cookies else '❌ Tidak digunakan'}")
print(f"- 📄 Metadata JSON   : {'✅ Tersimpan' if info else '❌ Tidak ada'}")
print(f"- 📁 Total Video     : {len(downloaded)} file")
print(f"- 💾 Ukuran Total    : {total_size_mb:.2f} MB")
print(f"- 🕒 Selesai pada     : {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"- 📂 Lokasi Video    : {video_dir}")
print(f"- 📜 Daftar File     :")
for i, fname in enumerate(file_names, 1):
    print(f"   {i}. {fname}")

# Upload to Gogle Drive

In [ ]:
# @title 🔌 [Mount] Sambungkan Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title ☁️ [Manual] Upload ke Google Drive

drive_target_folder = ""  # @param {type:"string"}
drive_filename = ""  # @param {type:"string"}
file_to_upload = ""  # @param {type:"string"}

import os, shutil, sys, datetime

# 🔐 Cek apakah Google Drive sudah di-mount
if not os.path.exists("/content/drive"):
    print("❌ Google Drive belum di-mount.")
    print("🔧 Jalankan dulu cell: `from google.colab import drive; drive.mount('/content/drive')`")
    sys.exit()

# 📂 Validasi file sumber
if not os.path.exists(file_to_upload):
    print(f"❌ File tidak ditemukan: {file_to_upload}")
    sys.exit()

# 📄 Info file sumber
file_size_mb = os.path.getsize(file_to_upload) / (1024 * 1024)
file_name_local = os.path.basename(file_to_upload)
print(f"🔍 File sumber ditemukan:")
print(f"   📄 Nama       : {file_name_local}")
print(f"   📦 Ukuran     : {file_size_mb:.2f} MB")
print(f"   📂 Lokasi     : {file_to_upload}")

# 📄 Gunakan nama asli jika kosong
if not drive_filename:
    drive_filename = file_name_local

# 📌 Bangun path lengkap tujuan
drive_target_path = os.path.join(drive_target_folder, drive_filename)

# 🛡️ Validasi path tujuan
if not drive_target_path.startswith("/content/drive/"):
    print("❌ Path tujuan harus berada di dalam `/content/drive/`")
    sys.exit()

# 📁 Buat folder jika belum ada
if not os.path.exists(drive_target_folder):
    os.makedirs(drive_target_folder)
    print(f"📁 Folder baru dibuat: {drive_target_folder}")
else:
    print(f"📁 Folder tujuan ditemukan: {drive_target_folder}")

# ☁️ Mulai proses upload
print("📤 Mengunggah file ke Google Drive...")
shutil.copy(file_to_upload, drive_target_path)

# 🕒 Info selesai
upload_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# ✅ Ringkasan akhir
print("\n✅ File berhasil dikirim ke Google Drive:")
print(f"   📄 Nama File   : {drive_filename}")
print(f"   📦 Ukuran      : {file_size_mb:.2f} MB")
print(f"   📂 Lokasi GDrive : {drive_target_path}")
print(f"   🕒 Selesai pada  : {upload_time}")

In [ ]:
# @title ☁️ [Manual] Upload Banyak File ke Google Drive (Auto Rename)
# @markdown - `upload_source_folder`: Folder lokal berisi file yang akan diupload

# @markdown - `drive_target_folder`: Tujuan upload di Google Drive

upload_source_folder = ""  # @param {type:"string"}
drive_target_folder = ""  # @param {type:"string"}

import os, shutil, sys, datetime

# 🔐 Cek Google Drive
if not os.path.exists("/content/drive"):
    print("❌ Google Drive belum di-mount.")
    print("🔧 Jalankan dulu cell: `from google.colab import drive; drive.mount('/content/drive')`")
    sys.exit()

# 📁 Cek folder sumber
if not os.path.exists(upload_source_folder):
    print(f"❌ Folder sumber tidak ditemukan: {upload_source_folder}")
    sys.exit()

# 🧾 Ambil daftar file (hanya file, bukan folder)
file_list = [f for f in os.listdir(upload_source_folder)
             if os.path.isfile(os.path.join(upload_source_folder, f))]

if not file_list:
    print("⚠️ Tidak ada file di dalam folder untuk diupload.")
    sys.exit()

# 📁 Buat folder tujuan jika belum ada
if not os.path.exists(drive_target_folder):
    os.makedirs(drive_target_folder)
    print(f"📁 Folder tujuan baru dibuat: {drive_target_folder}")
else:
    print(f"📁 Folder tujuan ditemukan: {drive_target_folder}")

# 🚀 Mulai upload
print(f"\n📤 Mengunggah {len(file_list)} file ke Google Drive...\n")

for i, filename in enumerate(file_list, 1):
    local_path = os.path.join(upload_source_folder, filename)

    # Rename otomatis jika file sudah ada
    name_only, ext = os.path.splitext(filename)
    drive_path = os.path.join(drive_target_folder, filename)
    counter = 1
    while os.path.exists(drive_path):
        new_name = f"{name_only}_{counter}{ext}"
        drive_path = os.path.join(drive_target_folder, new_name)
        counter += 1
    final_name = os.path.basename(drive_path)

    # Info ukuran
    file_size_mb = os.path.getsize(local_path) / (1024 * 1024)

    # Salin file
    shutil.copy(local_path, drive_path)
    upload_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Tampilkan status
    if counter > 1:
        print(f"⚠️  File dengan nama sama ditemukan. Diubah menjadi: {final_name}")
    print(f"✅ ({i}/{len(file_list)}) {final_name}")
    print(f"   📦 Ukuran   : {file_size_mb:.2f} MB")
    print(f"   📂 Tujuan   : {drive_path}")
    print(f"   🕒 Waktu    : {upload_time}\n")

print("🎉 Semua file berhasil diupload (dengan rename otomatis jika perlu).")

# Upload to Telegram

In [ ]:
# @title 🎯 Batch Ambil Metadata + Log Ganda

video_dir = ""  # @param {type:"string"}

import os, subprocess, json
from glob import glob
from datetime import datetime

# 📁 Inisialisasi direktori
base_dir = "/content/video_downloader"
meta_dir = os.path.join(base_dir, "meta")
thumb_dir = os.path.join(base_dir, "thumbnails")
logs_dir = os.path.join(base_dir, "logs")
os.makedirs(meta_dir, exist_ok=True)
os.makedirs(thumb_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

log_txt_path = os.path.join(logs_dir, "log.txt")
log_json_path = os.path.join(logs_dir, "log.json")

# ✏️ Fungsi log ke txt
def tulis_log_txt(teks):
    with open(log_txt_path, "a", encoding="utf-8") as f:
        f.write(teks.strip() + "\n")

# ✏️ Fungsi log ke json
def tulis_log_json(entry):
    try:
        if os.path.exists(log_json_path):
            with open(log_json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
        else:
            data = []
    except:
        data = []
    data.append(entry)
    with open(log_json_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

# 🔍 Ambil metadata dari 1 file
def extract_video_info(path, thumbnail_path):
    result = subprocess.run(
        [
            "ffprobe", "-v", "error",
            "-show_entries",
            "format=duration,format_name,size,bit_rate:"
            "stream=index,codec_type,codec_name,width,height,r_frame_rate,avg_frame_rate,bit_rate,channels,sample_rate,pix_fmt,profile",
            "-of", "json", path
        ],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )

    if result.returncode != 0:
        raise RuntimeError(f"ffprobe gagal:\n{result.stderr}")

    data = json.loads(result.stdout)
    if "format" not in data:
        raise ValueError("Metadata 'format' tidak ditemukan.")

    fmt = data["format"]
    duration = float(fmt.get("duration", 0))
    duration_str = f"{int(duration // 60)}:{int(duration % 60):02d}"
    size_bytes = int(fmt.get("size", 0))
    format_name = fmt.get("format_name", "unknown")
    bit_rate = int(fmt.get("bit_rate", 0)) if "bit_rate" in fmt else None

    video_stream = next((s for s in data["streams"] if s["codec_type"] == "video"), {})
    audio_stream = next((s for s in data["streams"] if s["codec_type"] == "audio"), {})

    width = video_stream.get("width", 0)
    height = video_stream.get("height", 0)
    resolution = f"{width}x{height}" if width and height else "unknown"
    v_codec = video_stream.get("codec_name", "unknown")
    v_pix_fmt = video_stream.get("pix_fmt", "unknown")
    v_profile = video_stream.get("profile", "unknown")
    v_fps = video_stream.get("avg_frame_rate", "0/1")
    v_bitrate = int(video_stream.get("bit_rate", 0)) if "bit_rate" in video_stream else None

    a_codec = audio_stream.get("codec_name", "unknown")
    a_channels = audio_stream.get("channels", 0)
    a_sample_rate = int(audio_stream.get("sample_rate", 0)) if "sample_rate" in audio_stream else None
    a_bitrate = int(audio_stream.get("bit_rate", 0)) if "bit_rate" in audio_stream else None

    return {
        "video_path": path,
        "thumbnail_path": thumbnail_path,
        "filename": os.path.basename(path),
        "duration": int(duration),
        "duration_str": duration_str,
        "format": format_name,
        "size_mb": round(size_bytes / (1024 * 1024), 2),
        "bit_rate": bit_rate,
        "resolution": resolution,
        "width": width,
        "height": height,
        "video_codec": v_codec,
        "video_pix_fmt": v_pix_fmt,
        "video_profile": v_profile,
        "video_fps": v_fps,
        "video_bitrate": v_bitrate,
        "audio_codec": a_codec,
        "audio_channels": a_channels,
        "audio_sample_rate": a_sample_rate,
        "audio_bitrate": a_bitrate,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

# 🔁 Batch proses semua video
video_files = sorted(glob(os.path.join(video_dir, "*.*")))
processed = 0

for video_path in video_files:
    try:
        ext = os.path.splitext(video_path)[1].lower()
        if ext not in [".mp4", ".mkv", ".avi", ".mov", ".webm"]:
            print(f"⏩ Melewati non-video: {video_path}")
            continue

        basename = os.path.splitext(os.path.basename(video_path))[0]
        thumbnail_path = os.path.join(thumb_dir, f"{basename}_thumb.jpg")
        json_path = os.path.join(meta_dir, f"{basename}_meta.json")

        # 📸 Ambil thumbnail
        print(f"📸 Thumbnail: {basename}")
        subprocess.run([
            "ffmpeg", "-y",
            "-i", video_path,
            "-ss", "00:00:01.000",
            "-vframes", "1",
            "-q:v", "2",
            thumbnail_path
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # 📦 Ambil metadata & simpan
        metadata = extract_video_info(video_path, thumbnail_path)

        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(metadata, f, indent=2)

        # ✅ Log
        ringkas = f"[✅] {metadata['filename']} | {metadata['resolution']} | {metadata['video_codec']}/{metadata['audio_codec']} | {metadata['size_mb']}MB | {metadata['duration_str']}"
        tulis_log_txt(ringkas)
        tulis_log_json(metadata)

        # 📊 Ringkasan ke konsol
        print(f"""
✅ Metadata: {metadata['filename']}
🖼️ Thumbnail : {os.path.basename(thumbnail_path)}
⏱️ Durasi    : {metadata['duration_str']}
📐 Resolusi  : {metadata['resolution']}
🎥 Video     : {metadata['video_codec']}
🎧 Audio     : {metadata['audio_codec']}
💾 Ukuran    : {metadata['size_mb']} MB
📀 Format    : {metadata['format'].upper()}
""")

        processed += 1

    except Exception as e:
        err_filename = os.path.basename(video_path)
        print(f"❌ Gagal: {err_filename} | {e}")
        tulis_log_txt(f"[❌] {err_filename} | GAGAL - {str(e).splitlines()[0]}")
        tulis_log_json({
            "status": "error",
            "filename": err_filename,
            "error": str(e),
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })

print(f"\n📁 Total diproses: {processed} video\n")

In [ ]:
# @title ✈️ Batch Upload ke Telegram

meta_dir = "/content/video_downloader/meta"
logs_dir = "/content/video_downloader/logs"
os.makedirs(logs_dir, exist_ok=True)
log_txt = os.path.join(logs_dir, "log.txt")
log_json = os.path.join(logs_dir, "log.json")

CHAT_ID = 123      # @param {type:"number"}
API_ID = 123         # @param {type:"number"}
API_HASH = ""      # @param {type:"string"}
BOT_TOKEN = ""     # @param {type:"string"}

!pip install -U pyrogram tgcrypto

from pyrogram import Client
from pyrogram.enums import ParseMode
from tqdm import tqdm
import asyncio, json, os, glob, traceback, re, time

# 🔠 Markdown escape
def escape_md(text):
    return re.sub(r'([*\[\]~`>#+\-=|{}!])', r'\\\1', text)

# 📓 Tulis log.txt (ringkas)
def tulis_log_txt(teks):
    with open(log_txt, "a", encoding="utf-8") as f:
        f.write(teks.strip() + "\n")

# 📦 Tulis log.json (terstruktur)
def tulis_log_json(entry):
    try:
        if os.path.exists(log_json):
            with open(log_json, "r", encoding="utf-8") as f:
                data = json.load(f)
        else:
            data = []
    except:
        data = []

    data.append(entry)
    with open(log_json, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

# 📊 Progress
def progress(current, total):
    bar.update(current - bar.n)

# ⏳ Status awal
def status_awal(filename, filesize_mb, duration, current_index, total_count):
    return f"""🚀 Upload Video ({current_index}/{total_count})

╭────────── Detail Upload ──────────╮

📁 Nama     : {filename}
📦 Ukuran   : {filesize_mb:.2f} MB
🕒 Durasi   : {duration} detik
⏳ Status   : Mengunggah...

╰─────────────────────────────╯
"""

# ✅ Status sukses
def status_sukses(filename, current_index, total_count, waktu_upload, meta):
    return f"""✅ Upload Berhasil! ({current_index}/{total_count})

╭────────── Detail Upload ──────────╮

🎬 File     : {filename}
📐 Resolusi: {meta.get("resolution", "?")}
🎥 Video    : {meta.get("video_codec", "?")} ({meta.get("video_bitrate", "?")} bps)
🎧 Audio    : {meta.get("audio_codec", "?")} ({meta.get("audio_bitrate", "?")} bps)
📊 Bitrate  : {meta.get("bit_rate", "?")} bps
📦 Ukuran   : {meta.get("size_mb", 0):.2f} MB
🕒 Durasi   : {meta.get("duration", 0)} detik
📤 Status   : Sukses
🧹 Cleanup  : File dihapus otomatis
⏱️ Waktu    : {waktu_upload:.2f} detik

╰─────────────────────────────╯
"""

# ❌ Status gagal
def status_error(filename, error_text, current_index, total_count):
    return f"""❌ Upload Gagal! ({current_index}/{total_count})

╭────────── Detail Upload ──────────╮

📁 File     : {filename}
⚠️ Error    : {error_text}

╰─────────────────────────────╯
"""

# 📤 Upload per video
async def kirim_video(app, meta_path, current_index, total_count):
    try:
        start_upload = time.time()
        with open(meta_path, "r") as f:
            meta = json.load(f)

        video_path = meta["video_path"]
        thumbnail_path = meta["thumbnail_path"]
        filename = escape_md(meta["filename"])
        duration = meta["duration"]
        filesize = os.path.getsize(video_path)
        filesize_mb = filesize / (1024 * 1024)

        print(f"\n📤 Mengunggah video {current_index} dari {total_count}: {filename}")
        await app.send_message(
            chat_id=CHAT_ID,
            text=status_awal(filename, filesize_mb, duration, current_index, total_count),
            parse_mode=ParseMode.MARKDOWN
        )

        global bar
        bar = tqdm(total=filesize, unit='B', unit_scale=True, desc=f"📤 Upload {current_index}/{total_count}", dynamic_ncols=True)

        await app.send_video(
            chat_id=CHAT_ID,
            video=video_path,
            thumb=thumbnail_path,
            caption=filename,
            duration=duration,
            supports_streaming=True,
            progress=progress
        )
        bar.close()

        waktu_upload = round(time.time() - start_upload, 2)

        # ✅ Send message
        await app.send_message(
            chat_id=CHAT_ID,
            text=status_sukses(filename, current_index, total_count, waktu_upload, meta),
            parse_mode=ParseMode.MARKDOWN
        )

        # 📝 Tulis ke log
        plain_log = f"[✅] {meta['filename']} | {meta.get('resolution')} | {meta.get('video_codec')}/{meta.get('audio_codec')} | {meta.get('size_mb')}MB | {meta.get('duration')}s | {waktu_upload}s"
        tulis_log_txt(plain_log)

        json_log = {
            "status": "success",
            "filename": meta["filename"],
            "resolution": meta.get("resolution"),
            "video_codec": meta.get("video_codec"),
            "audio_codec": meta.get("audio_codec"),
            "duration": meta.get("duration"),
            "size_mb": meta.get("size_mb"),
            "bit_rate": meta.get("bit_rate"),
            "video_bitrate": meta.get("video_bitrate"),
            "audio_bitrate": meta.get("audio_bitrate"),
            "upload_time": waktu_upload,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
        tulis_log_json(json_log)

        # 🧹 Hapus file
        for f in [thumbnail_path, meta_path, video_path]:
            if os.path.exists(f):
                os.remove(f)

    except Exception as e:
        error_msg = str(e) or traceback.format_exc()
        err_filename = meta.get("filename", os.path.basename(meta_path))

        await app.send_message(
            chat_id=CHAT_ID,
            text=status_error(escape_md(err_filename), escape_md(error_msg), current_index, total_count),
            parse_mode=ParseMode.MARKDOWN
        )
        tulis_log_txt(f"[❌] {err_filename} | GAGAL - {error_msg.strip().splitlines()[0]}")
        tulis_log_json({
            "status": "error",
            "filename": err_filename,
            "error": error_msg.strip().splitlines()[0],
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        })

# 🔁 Batch upload semua file
async def batch_upload():
    async with Client("upload_bot", api_id=API_ID, api_hash=API_HASH, bot_token=BOT_TOKEN) as app:
        meta_files = sorted(glob.glob(os.path.join(meta_dir, "*_meta.json")))
        total = len(meta_files)
        print(f"📦 Menemukan {total} file untuk diunggah.")

        if total == 0:
            await app.send_message(
                chat_id=CHAT_ID,
                text="⚠️ Tidak ada file video yang ditemukan untuk diupload.",
                parse_mode=ParseMode.MARKDOWN
            )
            return

        start_time = time.time()
        total_size_bytes = sum(
            os.path.getsize(json.load(open(f))["video_path"]) for f in meta_files if os.path.exists(f)
        )

        for idx, meta_path in enumerate(meta_files, start=1):
            await kirim_video(app, meta_path, idx, total)
            await asyncio.sleep(2)

        elapsed = time.time() - start_time
        minutes, seconds = divmod(int(elapsed), 60)
        total_size_mb = total_size_bytes / (1024 * 1024)

        summary = f"""✅ Batch Upload Selesai!!

📁 Total File   : {total} video
📦 Total Ukuran : {total_size_mb:.2f} MB
⏱️ Total Waktu  : {minutes} menit {seconds} detik

🎉 Semua video berhasil diupload!
"""

        await app.send_message(chat_id=CHAT_ID, text=summary, parse_mode=ParseMode.MARKDOWN)
        tulis_log_txt(f"[📦] Batch selesai: {total} file, {total_size_mb:.2f} MB, {minutes}m {seconds}s")
        tulis_log_json({
            "status": "batch_done",
            "total_files": total,
            "total_size_mb": round(total_size_mb, 2),
            "elapsed_sec": int(elapsed),
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        })

# ▶️ Jalankan
await batch_upload()

# Utility

In [ ]:
# @title 📑 Tampilkan Logs File
# @markdown - Menampilkan isi file .json (biasa atau Whisper), .txt, .log, atau .srt
transcript_path = ""  # @param {type:"string"}

import os
import json

# 📂 Validasi path
if not os.path.exists(transcript_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {transcript_path}")
print(f"📑 Menampilkan isi dari: {transcript_path}\n")

# 📘 Format: Whisper JSON (daftar segment dengan waktu dan teks)
def tampilkan_whisper_json(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if isinstance(data, list):  # list of segments
        for seg in data:
            start = round(seg.get("start", 0), 2)
            end = round(seg.get("end", 0), 2)
            text = seg.get("text", "").strip()
            print(f"[{start} - {end}] {text}")
    else:
        print(json.dumps(data, indent=2, ensure_ascii=False))

# 📘 Format: SRT Subtitle
def tampilkan_srt(path):
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    block = []
    for line in lines:
        line = line.strip()
        if line == "":
            if len(block) >= 3:
                index = block[0]
                timecode = block[1]
                text = " ".join(block[2:])
                print(f"[{timecode}] {text}")
            block = []
        else:
            block.append(line)

# 📘 Format: JSON Biasa
def tampilkan_json_biasa(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(json.dumps(data, indent=2, ensure_ascii=False))

# 📘 Format: TXT atau LOG
def tampilkan_txt_log(path):
    with open(path, "r", encoding="utf-8") as f:
        print(f.read())

# 🔍 Deteksi & Tampilkan Sesuai Ekstensi
if transcript_path.endswith(".json"):
    try:
        # Coba cek apakah ini Whisper JSON (list of segments)
        with open(transcript_path, "r", encoding="utf-8") as f:
            content = json.load(f)
        if isinstance(content, list) and "start" in content[0] and "text" in content[0]:
            tampilkan_whisper_json(transcript_path)
        else:
            tampilkan_json_biasa(transcript_path)
    except Exception as e:
        print(f"❌ Gagal membaca JSON: {e}")

elif transcript_path.endswith(".srt"):
    tampilkan_srt(transcript_path)

elif transcript_path.endswith(".txt") or transcript_path.endswith(".log"):
    tampilkan_txt_log(transcript_path)

else:
    raise ValueError("❌ Format file tidak didukung. Gunakan .json, .srt, .txt, atau .log")

In [ ]:
# @title 🗂️ [Manual] Kompres Folder Menjadi .zip
# @markdown - Masukkan path folder yang akan dikompres, dan nama file zip output
folder_to_zip = ""  # @param {type:"string"}
zip_filename = ""  # @param {type:"string"}

import shutil
import os

# Lokasi penyimpanan ZIP di dalam video_downloader
output_dir = "/content/video_downloader/output"
os.makedirs(output_dir, exist_ok=True)  # Buat folder jika belum ada

zip_path = os.path.join(output_dir, zip_filename)

# Validasi folder input
if not os.path.exists(folder_to_zip):
    raise FileNotFoundError(f"❌ Folder tidak ditemukan: {folder_to_zip}")

# Kompres folder
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_to_zip)
print(f"✅ Folder berhasil dikompres ke ZIP: {zip_path}")

# Tampilkan isi folder yang dikompres
print("\n📂 Daftar isi dalam folder:")
for root, dirs, files in os.walk(folder_to_zip):
    for file in files:
        rel_dir = os.path.relpath(root, folder_to_zip)
        rel_file = os.path.join(rel_dir, file) if rel_dir != "." else file
        print(f"  ├─ {rel_file}")

# Tampilkan ukuran file zip
if os.path.exists(zip_path):
    size_mb = os.path.getsize(zip_path) / (1024 * 1024)
    print(f"\n📦 Ukuran file ZIP: {size_mb:.2f} MB")
else:
    print("⚠️ File ZIP tidak ditemukan.")

In [ ]:
# @title 🗂️ [Manual] Ekstrak file RAR
import os

input_path = ""  # @param {type:"string"}
output_path = "/content/video_downloader/output"
os.makedirs(output_path, exist_ok=True)  # Buat folder jika belum ada

# Jalankan perintah rar
!unrar x "{input_path}" "{output_path}"

In [ ]:
# @title 🗂️ [Manual] Ekstrak file ZIP

input_path = ""  # @param {type:"string"}
output_path = "/content/video_downloader/output"
os.makedirs(output_path, exist_ok=True)  # Buat folder jika belum ada

# Jalankan perintah unzip
!unzip "{input_path}" -d "{output_path}"